In [1]:
from google.colab import drive
import os
drive.mount('/content/gdrive')
os.chdir("/content/gdrive/Shared drives/Mango/root/")
os.getcwd()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


'/content/gdrive/Shared drives/Mango/root'

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [2]:
os.chdir("/content/gdrive/Shared drives/Mango/root/DeepLearning/") #更改路徑

import tensorflow as tf
import pandas as pds
import matplotlib.pyplot as plt
from IPython import display
from PIL import Image

# 定義方法包list
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


def _bytes_feature(value):
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy()
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


def make_dataset(fileName, folderName):
    # read csv存放每張圖片的分類等級
    # filename=test .csv
    filedata = pds.read_csv(fileName + '.csv', encoding='big5')
    print(filedata)

    # 寫入tfrecord
    writer = tf.io.TFRecordWriter('train_' + fileName + '.tfrecords')

    for i in range(filedata.shape[0]):
        # filedata的維數　第一個代表column
        if filedata.iloc[i, 1].upper() == 'A':
            label = 0
            # 如果是A 那個example的label就是1
        elif filedata.iloc[i, 1].upper() == 'B':
            label = 1
        elif filedata.iloc[i, 1].upper() == 'C':
            label = 2

        img = Image.open(folderName + '/' + filedata.iloc[i, 0])
        # img= 把 C1-P1_Train/0000i.jpg 打開
        # resize成256*256
        img = img.resize((256, 256))
        img_raw = img.tobytes()
        # 解碼圖片

        # 每個 Example 含有 label 、 img_raw 兩個資訊。
        example = tf.train.Example(features=tf.train.Features(feature={
            "label": _int64_feature(label),
            "img_raw": _bytes_feature(img_raw)
        }))
        # 序列化為字串
        writer.write(example.SerializeToString())
    writer.close()


folderName = 'C1-P1_Train'  # 從C1-P1_Train拿圖片

make_dataset('first', folderName)
make_dataset('second', folderName)
make_dataset('third', folderName)

# classifier1
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.models import Sequential  # 順序式模型
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.utils import to_categorical

print('Classifier 1: ')
# 用feature_description包每一個圖片的label img_raw特徵資料
feature_description = {
    'label': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'img_raw': tf.io.FixedLenFeature([], tf.string, default_value=''),
}


def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


def _bytes_feature(value):
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy()
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


def _parse_function(example_proto):
    # 拿photo的feature
    return tf.io.parse_single_example(example_proto, feature_description)


def build_model():
    input_shape = (256, 256, 3)
    # 256*256*3
    model = Sequential()
    # 順序式模型

    model.add(Conv2D(64, kernel_size=(3, 3), input_shape=input_shape, padding='same', activation='relu'))
    # 用add添加層 input layer有64個資料(batchsize) 捲曲神經3*3 用input_shape傳遞圖片尺寸256*256*3
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', ))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same', ))
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same', ))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same', ))
    model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same', ))
    model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same', ))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same', ))
    model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same', ))
    model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same', ))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same', ))
    model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same', ))
    model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same', ))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    # 3個分類的模型
    # softmax:介於01間且機率總和=1 分成三類個別的機率
    # relu:忽略負值

    model.summary()
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
    # 定義損失函數loss->categorical_crossentropy多分類問題=>模型試圖最小化的目標函數 現有損失函數的字符串標識符
    # 預測值與實際值越相近 loss越小
    # 成效衡量指標metrics->默認的評估函數

    return model


def show_image(filename):
    # file= 讀入 TFRecords 檔
    # filename傳入資料處理器
    file = tf.data.TFRecordDataset(filename)
    # 用剛剛的parsefunction把features傳入 迭代file的feature們
    parsed_image_dataset = file.map(_parse_function)

    i = 0
    # 印出mango圖片
    for image_features in parsed_image_dataset:
        image = tf.io.decode_raw(image_features['img_raw'], tf.uint8)
        image = tf.reshape(image, [256, 256, 3])
        plt.imshow(image)
        plt.title(i)
        plt.show()
        i += 1


def training(filename):
    # Checkpoint路徑地址
    checkpoint_path = "training_1/MangoIsGood-{epoch:04d}.ckpt"
    # checkpoint_path = 在os的路徑
    checkpoint_dir = os.path.dirname(checkpoint_path)

    # 建立callback存放model weights
    # 每 10 個 epochs 都存一次
    cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                     save_weights_only=True,
                                                     verbose=1,
                                                     period=10)

    # read tfrecord
    file = tf.data.TFRecordDataset(filename)

    # parse 照片資料
    parsed_image_dataset = file.map(_parse_function)

    # 列表化資料們
    image_batch = parsed_image_dataset.batch(len(list(parsed_image_dataset)))

    # create 迭代
    data = iter(image_batch).next()

    model = build_model()
    # read 最後更新的 checkpoint
    latest = tf.train.latest_checkpoint(checkpoint_dir)
    if latest is not None:
        model.load_weights(latest)

    # 取資料來train
    train = []
    label = []

    for i in range(len(data['label'])):
        temp = tf.io.decode_raw(data['img_raw'][i], tf.uint8)
        train.append(tf.reshape(temp, [256, 256, 3]).numpy())
        label.append(data['label'][i])

    # 包成numpy陣列
    train = np.array(train)
    label = np.array(label)
    label = to_categorical(label, 3)

    # 設定batch_size epochs數 拿weight callback
    train = train.reshape(train.shape[0], 256, 256, 3)
    history = model.fit(train, label, batch_size=64, epochs=10, verbose=2, callbacks=[cp_callback])

    # 保存模型
    model.save('my_model_1.h5')
    print(model.predict(train))

    # 印出機率
    predict_first = np.argmax(model.predict(train), axis=1)

    # 印出最大機率的label
    print(predict_first)

    # 先傳預設的圖片進去測試訓練結果回傳
    return predict_first


fileName = 'first'
p1 = training('train_first.tfrecords')
show_image('train_first.tfrecords')


# classifier2
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.models import Sequential  # 順序式模型
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.utils import to_categorical
print('Classifier 2: ')
# 用feature_description包每一個圖片的label img_raw特徵資料
feature_description = {
    'label': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'img_raw': tf.io.FixedLenFeature([], tf.string, default_value=''),
}


def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


def _bytes_feature(value):
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy()
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


def _parse_function(example_proto):
    # 拿photo的feature
    return tf.io.parse_single_example(example_proto, feature_description)


def build_model():
    input_shape = (256, 256, 3)
    # 256*256*3
    model = Sequential()
    # 順序式模型

    model.add(Conv2D(64, kernel_size=(3, 3), input_shape=input_shape, padding='same', activation='relu'))
    # 用add添加層 input layer有64個資料(batchsize) 捲曲神經3*3 用input_shape傳遞圖片尺寸256*256*3
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', ))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same', ))
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same', ))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same', ))
    model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same', ))
    model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same', ))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same', ))
    model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same', ))
    model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same', ))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same', ))
    model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same', ))
    model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same', ))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    # 3個分類的模型
    # softmax:介於01間且機率總和=1 分成三類個別的機率
    # relu:忽略負值

    model.summary()
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
    # 定義損失函數loss->categorical_crossentropy多分類問題=>模型試圖最小化的目標函數 現有損失函數的字符串標識符
    # 預測值與實際值越相近 loss越小
    # 成效衡量指標metrics->默認的評估函數

    return model


def show_image(filename):
    # file= 讀入 TFRecords 檔
    # filename傳入資料處理器
    file = tf.data.TFRecordDataset(filename)
    # 用剛剛的parsefunction把features傳入 迭代file的feature們
    parsed_image_dataset = file.map(_parse_function)

    i = 0
    # 印出mango圖片
    for image_features in parsed_image_dataset:
        image = tf.io.decode_raw(image_features['img_raw'], tf.uint8)
        image = tf.reshape(image, [256, 256, 3])
        plt.imshow(image)
        plt.title(i)
        plt.show()
        i += 1


def training(filename):
    # Checkpoint路徑地址
    checkpoint_path = "training_1/MangoIsGood-{epoch:04d}.ckpt"
    # checkpoint_path = 在os的路徑
    checkpoint_dir = os.path.dirname(checkpoint_path)

    # 建立callback存放model weights
    # 每 10 個 epochs 都存一次
    cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                     save_weights_only=True,
                                                     verbose=1,
                                                     period=10)

    # read tfrecord
    file = tf.data.TFRecordDataset(filename)

    # parse 照片資料
    parsed_image_dataset = file.map(_parse_function)

    # 列表化資料們
    image_batch = parsed_image_dataset.batch(len(list(parsed_image_dataset)))

    # create 迭代
    data = iter(image_batch).next()

    model = build_model()
    # read 最後更新的 checkpoint
    latest = tf.train.latest_checkpoint(checkpoint_dir)
    if latest is not None:
        model.load_weights(latest)

    # 取資料來train
    train = []
    label = []

    for i in range(len(data['label'])):
        temp = tf.io.decode_raw(data['img_raw'][i], tf.uint8)
        train.append(tf.reshape(temp, [256, 256, 3]).numpy())
        label.append(data['label'][i])

    # 包成numpy陣列
    train = np.array(train)
    label = np.array(label)
    label = to_categorical(label, 3)

    # 設定batch_size epochs數 拿weight callback
    train = train.reshape(train.shape[0], 256, 256, 3)
    history = model.fit(train, label, batch_size=64, epochs=10, verbose=2, callbacks=[cp_callback])

    # 保存模型
    model.save('my_model_2.h5')
    print(model.predict(train))

    # 印出機率
    predict_second = np.argmax(model.predict(train), axis=1)

    # 印出最大機率的label
    print(predict_second)

    # 先傳預設的圖片進去測試訓練結果回傳
    return predict_second


fileName = 'second'
p2 = training('train_second.tfrecords')
show_image('train_second.tfrecords')

# classifier3
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.models import Sequential  # 順序式模型
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.utils import to_categorical
print('Classifier 3: ')
# 用feature_description包每一個圖片的label img_raw特徵資料
feature_description = {
    'label': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'img_raw': tf.io.FixedLenFeature([], tf.string, default_value=''),
}


def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


def _bytes_feature(value):
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy()
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


def _parse_function(example_proto):
    # 拿photo的feature
    return tf.io.parse_single_example(example_proto, feature_description)


def build_model():
    input_shape = (256, 256, 3)
    # 256*256*3
    model = Sequential()
    # 順序式模型

    model.add(Conv2D(64, kernel_size=(3, 3), input_shape=input_shape, padding='same', activation='relu'))
    # 用add添加層 input layer有64個資料(batchsize) 捲曲神經3*3 用input_shape傳遞圖片尺寸256*256*3
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', ))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same', ))
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same', ))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same', ))
    model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same', ))
    model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same', ))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same', ))
    model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same', ))
    model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same', ))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same', ))
    model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same', ))
    model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same', ))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    # 3個分類的模型
    # softmax:介於01間且機率總和=1 分成三類個別的機率
    # relu:忽略負值

    model.summary()
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
    # 定義損失函數loss->categorical_crossentropy多分類問題=>模型試圖最小化的目標函數 現有損失函數的字符串標識符
    # 預測值與實際值越相近 loss越小
    # 成效衡量指標metrics->默認的評估函數

    return model


def show_image(filename):
    # file= 讀入 TFRecords 檔
    # filename傳入資料處理器
    file = tf.data.TFRecordDataset(filename)
    # 用剛剛的parsefunction把features傳入 迭代file的feature們
    parsed_image_dataset = file.map(_parse_function)

    i = 0
    # 印出mango圖片
    for image_features in parsed_image_dataset:
        image = tf.io.decode_raw(image_features['img_raw'], tf.uint8)
        image = tf.reshape(image, [256, 256, 3])
        plt.imshow(image)
        plt.title(i)
        plt.show()
        i += 1


def training(filename):
    # Checkpoint路徑地址
    checkpoint_path = "training_1/MangoIsGood-{epoch:04d}.ckpt"
    # checkpoint_path = 在os的路徑
    checkpoint_dir = os.path.dirname(checkpoint_path)

    # 建立callback存放model weights
    # 每 10 個 epochs 都存一次
    cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                     save_weights_only=True,
                                                     verbose=1,
                                                     period=10)

    # read tfrecord
    file = tf.data.TFRecordDataset(filename)

    # parse 照片資料
    parsed_image_dataset = file.map(_parse_function)

    # 列表化資料們
    image_batch = parsed_image_dataset.batch(len(list(parsed_image_dataset)))

    # create 迭代
    data = iter(image_batch).next()

    model = build_model()
    # read 最後更新的 checkpoint
    latest = tf.train.latest_checkpoint(checkpoint_dir)
    if latest is not None:
        model.load_weights(latest)

    # 取資料來train
    train = []
    label = []

    for i in range(len(data['label'])):
        temp = tf.io.decode_raw(data['img_raw'][i], tf.uint8)
        train.append(tf.reshape(temp, [256, 256, 3]).numpy())
        label.append(data['label'][i])

    # 包成numpy陣列
    train = np.array(train)
    label = np.array(label)
    label = to_categorical(label, 3)

    # 設定batch_size epochs數 拿weight callback
    train = train.reshape(train.shape[0], 256, 256, 3)
    history = model.fit(train, label, batch_size=64, epochs=10, verbose=2, callbacks=[cp_callback])

    # 保存模型
    model.save('my_model_3.h5')
    print(model.predict(train))

    # 印出機率
    predict_third = np.argmax(model.predict(train), axis=1)

    # 印出最大機率的label
    print(predict_third)

    # 先傳預設的圖片進去測試訓練結果回傳
    return predict_third


fileName = 'third'
p3 = training('train_third.tfrecords')
show_image('train_third.tfrecords')


# 讀demo圖片 PART 1
import tensorflow as tf
import numpy as np
import pandas as pds
import matplotlib.pyplot as plt
from IPython import display
from PIL import Image

feature_description = {
    'label': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'img_raw': tf.io.FixedLenFeature([], tf.string, default_value=''),
}


def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


def _bytes_feature(value):
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy()
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


def _parse_function(example_proto):
    # 拿photo的feature
    return tf.io.parse_single_example(example_proto, feature_description)


def make_dataset(fileName, folderName):
    #  寫入tfrecord
    writer = tf.io.TFRecordWriter('test_' + fileName + '.tfrecords')

    for i in range(30):
        label = -1
        img = Image.open(folderName + '/' + str(i + 1) + '.jpg')
        # img= 把 C1-P1_Test/i.jpg 打開
        img = img.resize((256, 256))
        img_raw = img.tobytes()
        # 解碼圖片

        # 每個 Example 含有 label 、 img_raw 兩個資訊
        example = tf.train.Example(features=tf.train.Features(feature={
            "label": _int64_feature(label),
            "img_raw": _bytes_feature(img_raw)
        }))
        # 序列化為字串
        writer.write(example.SerializeToString())
    writer.close()
    # print(example)


fileName = 'test1'  # 寫入tfrecord的資料檔名
folderName = 'whf'  # 要打開的資料夾拿圖片

make_dataset(fileName, folderName)


def getTFRecordsData(filename):
    # 讀 tfrecord
    file = tf.data.TFRecordDataset(filename)

    # 迭代照片資料
    parsed_image_dataset = file.map(_parse_function)

    # 用list包起照片資料
    image_batch = parsed_image_dataset.batch(len(list(parsed_image_dataset)))

    return image_batch


def firstpredictLabel_1():
    size = 256
    # 拿tfrecord
    image_batch = getTFRecordsData('test_test1.tfrecords')

    # create 迭代
    data = iter(image_batch).next()

    image = []
    for i in range(len(data['label'])):
        temp = tf.io.decode_raw(data['img_raw'][i], tf.uint8)
        image.append(tf.reshape(temp, [size, size, 3]).numpy())
    image = np.array(image)
    image = image.reshape(image.shape[0], size, size, 3)

    # 使用我們的classifier1
    model = tf.keras.models.load_model('my_model_1.h5')
    # 存預測結果1
    predict1 = np.argmax(model.predict(image), axis=1)
    print(predict1)
    return predict1


def firstpredictLabel_2():
    size = 256
    # 拿tfrecord
    image_batch = getTFRecordsData('test_test1.tfrecords')

    # create迭代
    data = iter(image_batch).next()

    image = []
    for i in range(len(data['label'])):
        temp = tf.io.decode_raw(data['img_raw'][i], tf.uint8)
        image.append(tf.reshape(temp, [size, size, 3]).numpy())
    image = np.array(image)
    image = image.reshape(image.shape[0], size, size, 3)

    # 使用我們的classifier2
    model = tf.keras.models.load_model('my_model_2.h5')
    # 存預測結果2
    predict2 = np.argmax(model.predict(image), axis=1)
    print(predict2)
    return predict2


def firstpredictLabel_3():
    size = 256
    # 拿tfrecord
    image_batch = getTFRecordsData('test_test1.tfrecords')

    # create迭代
    data = iter(image_batch).next()

    image = []
    for i in range(len(data['label'])):
        temp = tf.io.decode_raw(data['img_raw'][i], tf.uint8)
        image.append(tf.reshape(temp, [size, size, 3]).numpy())
    image = np.array(image)
    image = image.reshape(image.shape[0], size, size, 3)
    # 使用我們的classifier3
    model = tf.keras.models.load_model('my_model_3.h5')
    # 存預測結果3
    predict3 = np.argmax(model.predict(image), axis=1)
    print(predict3)
    return predict3


# 讀demo圖片 PART 2
import tensorflow as tf
import numpy as np
import pandas as pds
import matplotlib.pyplot as plt
from IPython import display
from PIL import Image

feature_description = {
    'label': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'img_raw': tf.io.FixedLenFeature([], tf.string, default_value=''),
}


def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


def _bytes_feature(value):
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy()
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


def _parse_function(example_proto):
    # 拿photo的feature
    return tf.io.parse_single_example(example_proto, feature_description)


def make_dataset(fileName, folderName):
    # 寫入tfrecord
    writer = tf.io.TFRecordWriter('test_' + fileName + '.tfrecords')

    for i in range(30):
        label = -1
        img = Image.open(folderName + '/' + str(i + 31) + '.jpg')
        # img= 把 C1-P1_Test/i.jpg 打開
        img = img.resize((256, 256))
        img_raw = img.tobytes()
        # 解碼圖片

        # 每個 Example 含有 label 、 img_raw 兩個資訊
        example = tf.train.Example(features=tf.train.Features(feature={
            "label": _int64_feature(label),
            "img_raw": _bytes_feature(img_raw)
        }))
        # 序列化為字串
        writer.write(example.SerializeToString())
    writer.close()
    # print(example)


fileName = 'test2'  # 寫入tfrecord的資料檔名
folderName = 'whf'  # 要打開的資料夾拿圖片

make_dataset(fileName, folderName)


def getTFRecordsData(filename):
    # 讀 tfrecord
    file = tf.data.TFRecordDataset(filename)

    # 迭代照片資料
    parsed_image_dataset = file.map(_parse_function)

    # 用list包起照片資料
    image_batch = parsed_image_dataset.batch(len(list(parsed_image_dataset)))

    return image_batch


def secondpredictLabel_1():
    size = 256
    # 拿tfrecord
    image_batch = getTFRecordsData('test_test2.tfrecords')

    # create 迭代
    data = iter(image_batch).next()

    image = []
    for i in range(len(data['label'])):
        temp = tf.io.decode_raw(data['img_raw'][i], tf.uint8)
        image.append(tf.reshape(temp, [size, size, 3]).numpy())
    image = np.array(image)
    image = image.reshape(image.shape[0], size, size, 3)

    # 使用我們的classifier1
    model = tf.keras.models.load_model('my_model_1.h5')
    # 存預測結果1
    predict1 = np.argmax(model.predict(image), axis=1)
    print(predict1)
    return predict1


def secondpredictLabel_2():
    size = 256
    # 拿tfrecord
    image_batch = getTFRecordsData('test_test2.tfrecords')

    # create迭代
    data = iter(image_batch).next()

    image = []
    for i in range(len(data['label'])):
        temp = tf.io.decode_raw(data['img_raw'][i], tf.uint8)
        image.append(tf.reshape(temp, [size, size, 3]).numpy())
    image = np.array(image)
    image = image.reshape(image.shape[0], size, size, 3)

    # 使用我們的classifier2
    model = tf.keras.models.load_model('my_model_2.h5')
    # 存預測結果2
    predict2 = np.argmax(model.predict(image), axis=1)
    print(predict2)
    return predict2


def secondpredictLabel_3():
    size = 256
    # 拿tfrecord
    image_batch = getTFRecordsData('test_test2.tfrecords')

    # create迭代
    data = iter(image_batch).next()

    image = []
    for i in range(len(data['label'])):
        temp = tf.io.decode_raw(data['img_raw'][i], tf.uint8)
        image.append(tf.reshape(temp, [size, size, 3]).numpy())
    image = np.array(image)
    image = image.reshape(image.shape[0], size, size, 3)
    # 使用我們的classifier3
    model = tf.keras.models.load_model('my_model_3.h5')
    # 存預測結果3
    predict3 = np.argmax(model.predict(image), axis=1)
    print(predict3)
    return predict3

#如果只用classifier1
print('answer:')
a1=firstpredictLabel_1()
b1=secondpredictLabel_1()



single=np.hstack((a1,b1))
a=np.array([single])
b=single+1
# 輸出成 CSV 檔
data={
    'ImageID':range(1,61),
    'PredictedLabel':b
}
csv=pds.DataFrame(data)
csv.to_csv('answer.csv',index=False)

#ensemble learning
print('Ensemble learning: ')
print('answer:')
a1=firstpredictLabel_1()
a2=firstpredictLabel_2()
a3=firstpredictLabel_3()
ave = (a1 + a2 +a3) / 3
print(ave)
first = np.ndarray.round(ave)
first = first.astype(int)
print(first)
b1=secondpredictLabel_1()
b2=secondpredictLabel_2()
b3=secondpredictLabel_3()
bve = (b1 + b2 +b3) / 3
print(bve)
second = np.ndarray.round(bve)
second = second.astype(int)
print(second)
ans=np.hstack((first,second))
# 輸出成 CSV 檔
data={
    'ImageID':range(1,61),
    'PredictedLabel':ans
}
csv=pds.DataFrame(data)
csv.to_csv('answer.csv',index=False)



Output hidden; open in https://colab.research.google.com to view.

In [ ]:
#Train generator
!python3 ./DCGAN/GanA/train.py \
--dataset 'folder' \
--dataroot "./DCGAN/GanA/train_data" \
--outf "./DCGAN/GanA/model_checkpoint" \
--OutRealImg "./DCGAN/GanA/real_image" \
--OutFakeImg "./DCGAN/GanA/fake_image" \
--cuda \
--niter 10 \
--workers 64 \
--netG "./DCGAN/GanA/model_checkpoint/netG_epoch_11.pth" \
--netD "./DCGAN/GanA/model_checkpoint/netD_epoch_11.pth" \

python3: can't open file './DCGAN/GanA/train.py': [Errno 2] No such file or directory


In [ ]:
# Gans
# GanA_1
os.chdir("/content/gdrive/Shared drives/Mango/DCGAN/GanA/")  # Comment this line if you have the files in the same folder with this code
!python3 ./GGA.py \
--dataset 'folder' --dataroot "./allDATA/" \
--outf "./A_Outf/" \
--OutRealImg "./REAL/" \
--OutFakeImg "./FAKE/" \
--cuda --niter 1 \
--workers 1 \
--batchSize 1 \
--netG "./A_Outf/netG_epoch_99.pth" \
--netD "./A_Outf/netD_epoch_99.pth" \
#Miyi
!python3 ./GANmainA.py \
--dataset 'folder' --dataroot "/content/gdrive/Shared drives/Mango/DCGAN/GanA/allDATA" \
--outf "/content/gdrive/Shared drives/Mango/DCGAN/GanA/A_Outf/" \
--OutRealImg "/content/gdrive/Shared drives/Mango/DCGAN/GanA/REAL/" \
--OutFakeImg "/content/gdrive/Shared drives/Mango/DCGAN/GanA/FAKE/" \
--cuda --niter 100 \
--workers 100 \
--netG "/content/gdrive/Shared drives/Mango/DCGAN/GanA/A_Outf/netG_epoch_207.pth"

!python ./DCGAN/GanA/GANmainA.py --dataset 'cifar10' --dataroot "/content/gdrive/Shared drives/Mango/CartoonGan/sample_image/" --outf "/content/gdrive/Shared drives/Mango/DCGAN/GanA/A_Outf/" --OutRealImg "/content/gdrive/Shared drives/Mango/DCGAN/GanA/REAL/" --OutFakeImg "/content/gdrive/Shared drives/Mango/DCGAN/GanA/FAKE/" --cuda --niter 25

# GanB
os.chdir("/content/gdrive/Shared drives/Mango/DCGAN/GanB/") # Comment this line if you have the files in the same folder with this code
#Andy
!python3 GANmainB.py \
--dataset 'folder' --dataroot "./allDATA/" \
--outf "./B_Outf/" \
--OutRealImg "./REAL/" \
--OutFakeImg "./FAKE/" \
--cuda --niter 100 \
--workers 64 \
--netG "./B_Outf/netG_epoch_98.pth" \
--netD "./B_Outf/netD_epoch_97.pth" \
--lr 0.0001

#GanC
os.chdir("/content/gdrive/Shared drives/Mango/DCGAN/GanC/") # Comment this line if you have the files in the same folder with this code
#Andy
!python3 GANmainC.py \
--dataset 'folder' --dataroot "./allDATA/" \
--outf "./C_Outf/" \
--OutRealImg "./REAL/" \
--OutFakeImg "./FAKE/" \
--cuda --niter 103 \
--workers 512 \
--netG "./C_Outf/netG_epoch_66.pth" \
--netD "./C_Outf/netD_epoch_66.pth" \
--lr 0.0002 \
--batchSize 1 \

FileNotFoundError: ignored

In [ ]:
#讀GAN A to train model 1 2 3
import tensorflow as tf
import numpy as np
import pandas as pds
import matplotlib.pyplot as plt
from IPython import display
from PIL import Image

print('Read Gan A to train models 1, 2, 3: ')
feature_description = {
    'label': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'img_raw': tf.io.FixedLenFeature([], tf.string, default_value=''),
}

def _int64_feature(value):
    return tf.train.Feature(int64_list = tf.train.Int64List(value = [value]))

def _bytes_feature(value):
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() 
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _parse_function(example_proto):
    #拿photo的feature
    return tf.io.parse_single_example(example_proto, feature_description)


def make_dataset(fileName, folderName):
    
    # 寫入tfrecord
    writer = tf.io.TFRecordWriter(fileName + '.tfrecords')

    for i in range(30):
        label=-1
        img = Image.open(folderName + '/' + str(i+1) +'.png')
        #img= 把 A/i.png 打開
        img = img.resize((256, 256))
        img_raw = img.tobytes()
        #解碼圖片

        # 每個 Example 含有 label 、 img_raw 兩個資訊
        example = tf.train.Example(features = tf.train.Features(feature = {
            "label": _int64_feature(label),
            "img_raw": _bytes_feature(img_raw)
        }))
         # 序列化為字串
        writer.write(example.SerializeToString())
    writer.close()
    #print(example)

fileName = 'A' 
folderName = 'A' 

make_dataset(fileName, folderName)

def getTFRecordsData(filename):
    # 讀 tfrecord
    file = tf.data.TFRecordDataset(filename)

    # 迭代照片資料
    parsed_image_dataset = file.map(_parse_function)

    # 用list包起照片資料
    image_batch = parsed_image_dataset.batch(len(list(parsed_image_dataset)))

    return image_batch

    
def GANA1():
    size = 256
    # 拿tfrecord
    image_batch = getTFRecordsData(fileName+'.tfrecords')

    # create 迭代
    data = iter(image_batch).next()

    image = []
    for i in range(len(data['label'])):
        temp = tf.io.decode_raw(data['img_raw'][i], tf.uint8)
        image.append(tf.reshape(temp, [size, size, 3]).numpy())
    image = np.array(image) 
    image = image.reshape(image.shape[0], size, size, 3)
    
    # 使用我們的classifier1
    model = tf.keras.models.load_model('my_model_1.h5')
    # 存預測結果1
    A1=np.argmax(model.predict(image), axis=1)
    print(A1)
    return A1

def GANA2():
    size = 256
    # 拿tfrecord
    image_batch = getTFRecordsData(fileName+'.tfrecords')

    # create迭代
    data = iter(image_batch).next()

    image = []
    for i in range(len(data['label'])):
        temp = tf.io.decode_raw(data['img_raw'][i], tf.uint8)
        image.append(tf.reshape(temp, [size, size, 3]).numpy())
    image = np.array(image) 
    image = image.reshape(image.shape[0], size, size, 3)

    # 使用我們的classifier2
    model = tf.keras.models.load_model('my_model_2.h5')
    # 存預測結果2
    A2=np.argmax(model.predict(image), axis=1)
    print(A2)
    return A2

def GANA3():
    size = 256
    # 拿tfrecord
    image_batch = getTFRecordsData(fileName+'.tfrecords')

    # create迭代
    data = iter(image_batch).next()

    image = []
    for i in range(len(data['label'])):
        temp = tf.io.decode_raw(data['img_raw'][i], tf.uint8)
        image.append(tf.reshape(temp, [size, size, 3]).numpy())
    image = np.array(image) 
    image = image.reshape(image.shape[0], size, size, 3)
    # 使用我們的classifier3
    model = tf.keras.models.load_model('my_model_3.h5')
    # 存預測結果3
    A3=np.argmax(model.predict(image), axis=1)
    print(A3)
    return A3
print('answer:')
a=GANA1()

#讀GAN B to train model 1 2 3
import tensorflow as tf
import numpy as np
import pandas as pds
import matplotlib.pyplot as plt
from IPython import display
from PIL import Image

print('Read Gan B to train models 1, 2, 3: ')
feature_description = {
    'label': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'img_raw': tf.io.FixedLenFeature([], tf.string, default_value=''),
}

def _int64_feature(value):
    return tf.train.Feature(int64_list = tf.train.Int64List(value = [value]))

def _bytes_feature(value):
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() 
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _parse_function(example_proto):
    #拿photo的feature
    return tf.io.parse_single_example(example_proto, feature_description)


def make_dataset(fileName, folderName):
    
    # 寫入tfrecord
    writer = tf.io.TFRecordWriter(fileName + '.tfrecords')

    for i in range(30):
        label=-1
        img = Image.open(folderName + '/' + str(i+1) +'.png')
        #img= 把 B/i.png 打開
        img = img.resize((256, 256))
        img_raw = img.tobytes()
        #解碼圖片

        # 每個 Example 含有 label 、 img_raw 兩個資訊
        example = tf.train.Example(features = tf.train.Features(feature = {
            "label": _int64_feature(label),
            "img_raw": _bytes_feature(img_raw)
        }))
         # 序列化為字串
        writer.write(example.SerializeToString())
    writer.close()
    #print(example)

fileName = 'B' 
folderName = 'B' 

make_dataset(fileName, folderName)

def getTFRecordsData(filename):
    # 讀 tfrecord
    file = tf.data.TFRecordDataset(filename)

    # 迭代照片資料
    parsed_image_dataset = file.map(_parse_function)

    # 用list包起照片資料
    image_batch = parsed_image_dataset.batch(len(list(parsed_image_dataset)))

    return image_batch

    
def GANB1():
    size = 256
    # 拿tfrecord
    image_batch = getTFRecordsData(fileName+'.tfrecords')

    # create 迭代
    data = iter(image_batch).next()

    image = []
    for i in range(len(data['label'])):
        temp = tf.io.decode_raw(data['img_raw'][i], tf.uint8)
        image.append(tf.reshape(temp, [size, size, 3]).numpy())
    image = np.array(image) 
    image = image.reshape(image.shape[0], size, size, 3)
    
    # 使用我們的classifier1
    model = tf.keras.models.load_model('my_model_1.h5')
    # 存預測結果1
    B1=np.argmax(model.predict(image), axis=1)
    print(B1)
    return B1

def GANB2():
    size = 256
    # 拿tfrecord
    image_batch = getTFRecordsData(fileName+'.tfrecords')

    # create迭代
    data = iter(image_batch).next()

    image = []
    for i in range(len(data['label'])):
        temp = tf.io.decode_raw(data['img_raw'][i], tf.uint8)
        image.append(tf.reshape(temp, [size, size, 3]).numpy())
    image = np.array(image) 
    image = image.reshape(image.shape[0], size, size, 3)

    # 使用我們的classifier2
    model = tf.keras.models.load_model('my_model_2.h5')
    # 存預測結果2
    A2=np.argmax(model.predict(image), axis=1)
    print(B2)
    return B2

def GANB3():
    size = 256
    # 拿tfrecord
    image_batch = getTFRecordsData(fileName+'.tfrecords')

    # create迭代
    data = iter(image_batch).next()

    image = []
    for i in range(len(data['label'])):
        temp = tf.io.decode_raw(data['img_raw'][i], tf.uint8)
        image.append(tf.reshape(temp, [size, size, 3]).numpy())
    image = np.array(image) 
    image = image.reshape(image.shape[0], size, size, 3)
    # 使用我們的classifier3
    model = tf.keras.models.load_model('my_model_3.h5')
    # 存預測結果3
    B3=np.argmax(model.predict(image), axis=1)
    print(B3)
    return B3
b=GANB1()

#讀GAN C to train model 1 2 3
import tensorflow as tf
import numpy as np
import pandas as pds
import matplotlib.pyplot as plt
from IPython import display
from PIL import Image

print('Read Gan C to train models 1, 2, 3:')
feature_description = {
    'label': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'img_raw': tf.io.FixedLenFeature([], tf.string, default_value=''),
}

def _int64_feature(value):
    return tf.train.Feature(int64_list = tf.train.Int64List(value = [value]))

def _bytes_feature(value):
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() 
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _parse_function(example_proto):
    #拿photo的feature
    return tf.io.parse_single_example(example_proto, feature_description)


def make_dataset(fileName, folderName):
    
    # 寫入tfrecord
    writer = tf.io.TFRecordWriter(fileName + '.tfrecords')

    for i in range(30):
        label=-1
        img = Image.open(folderName + '/' + str(i+1) +'.png')
        #img= 把 C/i.png 打開
        img = img.resize((256, 256))
        img_raw = img.tobytes()
        #解碼圖片

        # 每個 Example 含有 label 、 img_raw 兩個資訊
        example = tf.train.Example(features = tf.train.Features(feature = {
            "label": _int64_feature(label),
            "img_raw": _bytes_feature(img_raw)
        }))
         # 序列化為字串
        writer.write(example.SerializeToString())
    writer.close()
    #print(example)

fileName = 'C' 
folderName = 'C' 

make_dataset(fileName, folderName)

def getTFRecordsData(filename):
    # 讀 tfrecord
    file = tf.data.TFRecordDataset(filename)

    # 迭代照片資料
    parsed_image_dataset = file.map(_parse_function)

    # 用list包起照片資料
    image_batch = parsed_image_dataset.batch(len(list(parsed_image_dataset)))

    return image_batch

    
def GANC1():
    size = 256
    # 拿tfrecord
    image_batch = getTFRecordsData(fileName+'.tfrecords')

    # create 迭代
    data = iter(image_batch).next()

    image = []
    for i in range(len(data['label'])):
        temp = tf.io.decode_raw(data['img_raw'][i], tf.uint8)
        image.append(tf.reshape(temp, [size, size, 3]).numpy())
    image = np.array(image) 
    image = image.reshape(image.shape[0], size, size, 3)
    
    # 使用我們的classifier1
    model = tf.keras.models.load_model('my_model_1.h5')
    # 存預測結果1
    C1=np.argmax(model.predict(image), axis=1)
    print(C1)
    return C1

def GANC2():
    size = 256
    # 拿tfrecord
    image_batch = getTFRecordsData(fileName+'.tfrecords')

    # create迭代
    data = iter(image_batch).next()

    image = []
    for i in range(len(data['label'])):
        temp = tf.io.decode_raw(data['img_raw'][i], tf.uint8)
        image.append(tf.reshape(temp, [size, size, 3]).numpy())
    image = np.array(image) 
    image = image.reshape(image.shape[0], size, size, 3)

    # 使用我們的classifier2
    model = tf.keras.models.load_model('my_model_2.h5')
    # 存預測結果2
    C2=np.argmax(model.predict(image), axis=1)
    print(C2)
    return C2

def GANC3():
    size = 256
    # 拿tfrecord
    image_batch = getTFRecordsData(fileName+'.tfrecords')

    # create迭代
    data = iter(image_batch).next()

    image = []
    for i in range(len(data['label'])):
        temp = tf.io.decode_raw(data['img_raw'][i], tf.uint8)
        image.append(tf.reshape(temp, [size, size, 3]).numpy())
    image = np.array(image) 
    image = image.reshape(image.shape[0], size, size, 3)
    # 使用我們的classifier3
    model = tf.keras.models.load_model('my_model_3.h5')
    # 存預測結果3
    C3=np.argmax(model.predict(image), axis=1)
    print(C3)
    return C3
c=GANC1()

Read Gan A to train models 1, 2, 3: 
answer:
[1 0 1 1 1 0 0 1 2 0 0 2 0 1 1 1 2 2 2 1 1 1 1 0 2 1 0 0 1 1]
Read Gan B to train models 1, 2, 3: 
[1 0 1 1 1 0 0 1 2 0 0 2 0 1 1 1 2 2 2 1 1 1 1 0 2 1 0 0 1 1]
Read Gan C to train models 1, 2, 3:
[1 0 1 1 1 0 0 1 2 0 0 2 0 1 1 1 2 2 2 1 1 1 1 0 2 1 0 0 1 1]
